In [1]:
#Pytorch
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

#Torchvision
import torchvision
from torchvision import datasets, models, transforms, utils
  
#Pytorch
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

#Torchvision
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader

#Image Processing
import matplotlib.pyplot as plt
from skimage import io, transform, color
import PIL
from PIL import Image

#Others
import sklearn.metrics
from sklearn.metrics import *
import numpy as np
import pandas as pd
import cv2
import time
import os
import copy
from model_summary import *
import pretrainedmodels
import tqdm
from tqdm import tqdm_notebook as tqdm
import warnings
warnings.filterwarnings("ignore")

## Dataloader

In [2]:
class dataset(Dataset):

    def __init__(self, csv_file, root_dir, transform=None):

        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.mask_dir = self.root_dir.replace('images','masks')
        
    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,self.data_frame.iloc[idx]['name'])
        image = Image.open(img_name)
        
        mask_name = os.path.join(self.mask_dir,self.data_frame.iloc[idx]['name'])
        mask = io.imread(mask_name)
        mask = np.array([mask,mask,mask]).transpose((1,2,0))
        mask = Image.fromarray(mask)

        label = self.data_frame.iloc[idx]['category']       

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)
    
        return {'image':image, 'category':label, 'mask':mask, 'name':self.data_frame.iloc[idx]['name']}
    

def get_dataloader(data_dir, train_csv_path, image_size, img_mean, img_std, batch_size=1):

    data_transforms = {
        'train': transforms.Compose([
            transforms.Resize(image_size),
            transforms.RandomHorizontalFlip(0.5),
            transforms.RandomAffine(translate=(0,0.2),degrees=15,shear=15),
            transforms.ToTensor(),
            transforms.Normalize(img_mean,img_std)
        ]),
        'valid': transforms.Compose([
            transforms.Resize(image_size),
            transforms.ToTensor(),
            transforms.Normalize(img_mean,img_std)
        ]),
        'test': transforms.Compose([
            transforms.Resize(image_size),
            transforms.ToTensor(),
            transforms.Normalize(img_mean,img_std)
        ])
    }

    image_datasets = {}
    dataloaders = {}
    dataset_sizes = {}

    for x in ['train', 'valid', 'test']:
        if x == 'test':
            bs = 1
            sh = False
        else:
            bs = batch_size
            sh = True
        image_datasets[x] = dataset(train_csv_path.replace('train',x),root_dir=data_dir,transform=data_transforms[x])
        dataloaders[x] = torch.utils.data.DataLoader(image_datasets[x], batch_size=bs,shuffle=sh, num_workers=8)    
        dataset_sizes[x] = len(image_datasets[x])

    device = torch.device("cuda:0")

    return dataloaders,dataset_sizes,image_datasets,device

# a,_,_,_ = get_dataloader('../Data/oxford_pets/sparse_images/','../CSV/oxford_pet_train.csv',(224,224),[0,0,0],[1,1,1])
# b = iter(a['train']).next()
# c = b['mask'].squeeze().numpy().transpose((1,2,0))
# plt.imshow(c)

## Model

In [3]:
# class StdUpsample(nn.Module):
#     def __init__(self, nin, nout):
#         super().__init__()
#         self.conv = nn.ConvTranspose2d(nin, nout, 2, stride=2)
#         self.bn = nn.BatchNorm2d(nout)
        
#     def forward(self, x): return self.bn(F.relu(self.conv(x)))
    
# class VGG_UNet(nn.Module):
#     def __init__(self,base):
#         super().__init__()
#         self.base = base
#         self.conv1 = nn.ConvTranspose2d(512, 256, 2, stride=2)
#         self.conv2 = nn.ConvTranspose2d(256, 256, 2, stride=2)
#         self.conv3 = nn.ConvTranspose2d(256, 1, 2, stride=2)
#         self.bn = nn.BatchNorm2d(256)
        
#     def forward(self,x):
#         x = self.base(x)
#         x = self.bn(F.relu(self.conv1(x)))
#         x = self.bn(F.relu(self.conv2(x)))
#         x = self.bn(F.relu(self.conv2(x)))
#         x = self.bn(F.relu(self.conv2(x)))
#         x = self.conv3(x)
#         return x

# v = models.vgg16_bn(pretrained=True)
# v1 = nn.Sequential(*list(v.children())[:-1])
# model = VGG_UNet(v1)

In [4]:
class SaveFeatures():
    features=None
    def __init__(self, m): self.hook = m.register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output): self.features = output
    def remove(self): self.hook.remove()

class UnetBlock(nn.Module):
    def __init__(self, up_in, x_in, n_out):
        super().__init__()
        up_out = x_out = n_out//2
        self.x_conv  = nn.Conv2d(x_in,  x_out,  1)
        self.tr_conv = nn.ConvTranspose2d(up_in, up_out, 2, stride=2)
        self.bn = nn.BatchNorm2d(n_out)
        
    def forward(self, up_p, x_p):
        up_p = self.tr_conv(up_p)
        x_p = self.x_conv(x_p)
        cat_p = torch.cat([up_p,x_p], dim=1)
        return self.bn(F.relu(cat_p))

class Unet34(nn.Module):
    def __init__(self, rn):
        super().__init__()
        self.rn = rn
        self.sfs = [SaveFeatures(rn[0][i]) for i in [12,22,32,42]]
        self.up1 = UnetBlock(512,512,256)
        self.up2 = UnetBlock(256,512,256)
        self.up3 = UnetBlock(256,256,256)
        self.up4 = UnetBlock(256,128,256)
        self.up5 = nn.ConvTranspose2d(256, 1, 2, stride=2)
        
    def forward(self,x):
        x = F.relu(self.rn(x))
        x = self.up1(x, self.sfs[3].features)
        x = self.up2(x, self.sfs[2].features)
        x = self.up3(x, self.sfs[1].features)
        x = self.up4(x, self.sfs[0].features)
        x = self.up5(x)
        return x[:,0]
    
    def close(self):
        for sf in self.sfs: sf.remove()

def build_model():
    v = models.vgg16_bn(pretrained=True)
    v1 = nn.Sequential(*list(v.children())[:-1])
    m = Unet34(v1)
    return m

In [8]:
def denorm_img(img_ten,img_mean,img_std):

    bz,nc,h,w = img_ten.shape
    output = []
    img_num = img_ten.numpy()
    
    for i in range(bz):
        
        img = img_ten[i].numpy().squeeze()
        
        img[0,:,:] = img[0,:,:]*img_std[0]
        img[1,:,:] = img[1,:,:]*img_std[1]
        img[2,:,:] = img[2,:,:]*img_std[2]

        img[0,:,:] = img[0,:,:] + img_mean[0]
        img[1,:,:] = img[1,:,:] + img_mean[1]
        img[2,:,:] = img[2,:,:] + img_mean[2]
        
        img = img.mean(axis=0)
        img[img>=0.2*img.max()] = 1
        img[img<0.2*img.max()] = 0
        
        output.append(img)
    
    output = np.array(output)
    return output

def get_IoU(pred, targs):
    return 2*(pred*targs).sum() / ((pred+targs).sum())# - (pred*targs).sum())
    

class unet_bc():
    def __init__(self):
        
        #Initialization
        self.data_dir =  '../Data/oxford_pets/sparse_images/'
        self.train_csv = '../CSV/oxford_pet_train.csv'
        self.num_epochs = 200
        self.input_shape = (224,224)#(640,512) #(640,512)#(224,224)#(640,384) (640,512)
        self.batch_size = 8
        self.img_mean = [0,0,0]
        self.img_std = [1,1,1]
        self.exp_name = 'Weights/unet_oxford'
        
        #Define the three models
        self.model = build_model()
        
        #Put them on the GPU
        self.model = self.model.cuda()
        #self.model.load_state_dict(torch.load(self.exp_name+'.pt'))

        
        #Get the dataloaders
        self.dataloaders,self.dataset_sizes,self.dataset,self.device = get_dataloader(self.data_dir,self.train_csv,\
                                                        self.input_shape,self.img_mean,self.img_std,self.batch_size)
        

        self.optimizer = optim.Adam(self.model.parameters(),lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
        #self.optimizer = optim.SGD(self.model.parameters(), lr=0.01, momentum=0.99)
        
        self.exp_lr_scheduler = lr_scheduler.StepLR(self.optimizer, step_size=20, gamma=0.1)

        
    def train(self):
        
        since = time.time()
        best_epoch_acc = 0.0
        best_epoch_iou = 0.0
        
        for epoch in range(self.num_epochs):
            print('Epoch {}/{}'.format(epoch, self.num_epochs - 1),flush=True)
            print('-' * 10,flush=True)

            # Each epoch has a training and validation phase
            for phase in ['train', 'valid']:
                if phase == 'train':
                    
                    #Set the models to training mode
                    self.exp_lr_scheduler.step()
                    self.model.train()
                
                else:
                    #Set the models to evaluation mode
                    self.model.eval()
                    
                #Keep a track of all the three loss
                running_loss = 0.0
                
                #Metrics : predictor auc and selector iou
                running_iou = 0
                
                #tqdm bar
                pbar = tqdm(total=self.dataset_sizes[phase])

                # Iterate over data.
                for sampled_batch in self.dataloaders[phase]:

                    inputs = sampled_batch['image']
                    labels = sampled_batch['category']
                    mask = torch.Tensor(denorm_img(sampled_batch['mask'],self.img_mean,self.img_std))
                    #import pdb;pdb.set_trace()                    
                    #Input needs to be float and labels long
                    inputs = inputs.float().to(self.device)
                    labels = labels.long().to(self.device)
                    mask = mask.to(self.device)
                    
                    # zero the parameter gradients
                    self.optimizer.zero_grad()
                
                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        
                        outputs = self.model(inputs)
                        
                        out4loss = outputs.view(-1)
                        mask4loss = mask.view(-1)
                        #import pdb;pdb.set_trace()
                        
                        #loss = nn.MSELoss()(F.sigmoid(out4loss),mask4loss)
                        loss = nn.BCEWithLogitsLoss()(out4loss,mask4loss)
                        #loss = F.cross_entropy(outputs,mask)
                        
                        # backward + optimize only if in training phase
                        if phase == 'train':
                            
                            loss.backward()
                            self.optimizer.step()
                                    
                    preds = out4loss
                    preds[preds>0] = 1
                    preds[preds<=0] = 0
                    #import pdb;pdb.set_trace()
                    #print(preds.mean(),mask.mean())
                    
                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_iou += get_IoU(out4loss,mask4loss) * inputs.size(0)
                    #print(running_iou)
                    

                    pbar.update(inputs.shape[0])
                pbar.close()

                epoch_loss = running_loss / self.dataset_sizes[phase]
                epoch_iou = running_iou / self.dataset_sizes[phase]
                
                print('{} Sel_Loss: {:.4f} IoU: {:.4f}'.format(
                    phase, epoch_loss, epoch_iou))

                # deep copy the model
                if phase == 'valid' and epoch_iou > best_epoch_iou:
                    best_epoch_iou = epoch_iou
                    torch.save(self.model.state_dict(),self.exp_name+'.pt')
                    
        time_elapsed = time.time() - since
        print('Training complete in {:.0f}m {:.0f}s'.format(
            time_elapsed // 60, time_elapsed % 60))
        print('Best IoU: {:4f}'.format(best_epoch_iou))

        torch.save(self.model.state_dict(),self.exp_name+'_final.pt')
        
        print('Training completed finally !!!!!')
        
        
    def test_model(self):
                
        self.model.load_state_dict(torch.load(self.exp_name+'.pt'))
        self.model.eval()
        
        mIoU = 0
        total = 0
        mode = 'test'

        image_list = []
        mask_list = []
        pred_list = []
        
        with torch.no_grad():
            for data in self.dataloaders[mode]:

                images = data['image'].to(self.device)
                mask = denorm_img(data['mask'],self.img_mean,self.img_std)
                
                preds = self.model(images)
                
                image_list.append(images.squeeze().cpu().numpy().transpose((1,2,0)))
                mask_list.append(mask.transpose((1,2,0)).squeeze())
                pred_list.append(preds.squeeze().cpu().numpy())
                
        return image_list,mask_list,pred_list
    
    def test_model_acc(self):
                
        self.predictor.load_state_dict(torch.load(self.exp_name+'_pred.pt'))
        self.predictor.eval()
        
        acc = 0
        total = 0
        mode = 'test'

        with torch.no_grad():
            for data in self.dataloaders[mode]:

                images = data['image']
                labels = data['category']
                
                images = images.to(self.device)
                labels = labels.to(self.device)
                
                output = self.predictor(images)
                _,out = torch.max(output,1)
                
                total += labels.size(0)
                acc += torch.sum(out==labels.data)

        print("mIoU:", 1.0*acc.double()/total)
    

In [9]:
u = unet_bc()

In [10]:
u.train()

Epoch 0/199
----------


train Sel_Loss: 0.5572 IoU: 0.6281


valid Sel_Loss: 0.4092 IoU: 0.7692
Epoch 1/199
----------


train Sel_Loss: 0.4761 IoU: 0.6592


valid Sel_Loss: 0.3538 IoU: 0.7887
Epoch 2/199
----------


...

valid Sel_Loss: 0.3257 IoU: 0.7001
Epoch 198/199
----------


train Sel_Loss: 0.3892 IoU: 0.6665


valid Sel_Loss: 0.3272 IoU: 0.6914
Epoch 199/199
----------


train Sel_Loss: 0.3874 IoU: 0.6692


valid Sel_Loss: 0.3232 IoU: 0.7151
Training complete in 188m 5s
Best IoU: 0.788712
Training completed finally !!!!!


In [ ]:
%debug

In [ ]:
a,b,c = u.test_model()

In [ ]:
plt.imshow(a[1])

In [ ]:
plt.imshow(b[1])

In [ ]:
plt.imshow(c[1])

In [ ]:
u.train()

In [ ]:
%debug

In [ ]:
_,_,_ = dci.get_cam()

In [ ]:
dci.train()

In [ ]:
!nvidia-smi

In [ ]:
# dci.test_model_acc()

In [ ]:
md,dl = dci.return_model()

In [ ]:
a = iter(dl).next()

m = denorm_img(a['mask'],[0.223, 0.231, 0.243],[0.266, 0.270, 0.274]).squeeze()
bm = denorm_img(a['bmask'],[0.223, 0.231, 0.243],[0.266, 0.270, 0.274]).squeeze()

In [ ]:
device = torch.device("cuda:0")

p = F.sigmoid(md(a['image'].to(device))).detach().cpu().numpy().squeeze()

In [ ]:
m.shape

In [ ]:
p.shape

In [ ]:
bm.shape

In [ ]:
plt.imshow(bm)

In [ ]:
plt.imshow(m)

In [ ]:
p_m = p*bm
p_m[p_m > p_m.mean() + p_m.std()] = 1
p_m[p_m < p_m.mean() + p_m.std()] = 0

In [ ]:
plt.imshow(p_m)

In [ ]:
p_m_t = torch.Tensor(p_m)
print(nn.L1Loss()(torch.Tensor(p_m),torch.zeros(p_m_t.shape)))

In [ ]:
p*bm

In [ ]:
plt.imshow(p)